In [1]:
import os
import re
import json
import chardet
import scr.public_function as pf

# NOTE 中间的所有文件都是UTF8
encoding = 'utf16'
output_encoding = 'utf16'

pf._init_()
name = '[110520][ディーゼルマイン] 君がいた図書室'

In [ ]:
# FIXME 从游戏原文本文件提取日文


def get_scenario_from_origin(data: list) -> list:
    text_all = []
    cnt = 0
    # START FIXME
    for line in data:
        if pf.has_jp(line) and line[0] not in "@*;\t /S[":
            text_all.append(line)
        cnt+=1
    # END
    return text_all

pf.extract_jp(pf.get_scenario_from_origin_sakura, encoding)


In [6]:
# FIXME 创建翻译字典
pf.create_dict(lambda x:x[:-1])

总共 19780 条
添加关键字： 150 条


In [ ]:
# 检查字典key数量是否正确
pf.check_dict()

In [7]:
# 把字典翻译

# print(format_yu_ris('歩「じゃあ≪射精／だ≫して……ね？　いっぱい……≪射精／だ≫してほしい」'))
pf.translate(delete_func=None)
# print(pf._translate('思う存分、[ruby text=な]膣[ruby text=か]内[ruby text=だ]射[ruby text=し]精して下さぁい[heart]」', delete_func=format_text))

30 / 18745
60 / 18745
90 / 18745
120 / 18745
150 / 18745
180 / 18745
210 / 18745
240 / 18745
270 / 18745
300 / 18745
330 / 18745
360 / 18745
390 / 18745
420 / 18745
450 / 18745
480 / 18745
510 / 18745
540 / 18745
570 / 18745
600 / 18745
630 / 18745
660 / 18745
690 / 18745
720 / 18745
750 / 18745
780 / 18745
810 / 18745
840 / 18745
870 / 18745
900 / 18745
930 / 18745
960 / 18745
990 / 18745
1020 / 18745
1050 / 18745
1080 / 18745
1110 / 18745
1140 / 18745
1170 / 18745
1200 / 18745
1230 / 18745
1260 / 18745
1290 / 18745
1320 / 18745
1350 / 18745
1380 / 18745
1410 / 18745
1440 / 18745
1470 / 18745
1500 / 18745
1530 / 18745
1560 / 18745
1590 / 18745
1620 / 18745
1650 / 18745
1680 / 18745
1710 / 18745
1740 / 18745
1770 / 18745
1800 / 18745
1830 / 18745
1860 / 18745
1890 / 18745
1920 / 18745
1950 / 18745
1980 / 18745
2010 / 18745
2040 / 18745
2070 / 18745
2100 / 18745
2130 / 18745
2160 / 18745
2190 / 18745
2220 / 18745
2250/ 18745
2280 / 18745
2310 / 18745
2340 / 18745
2370 / 18745
2400 / 187

In [ ]:
# 检查是否还有没翻译的语句
pf.check_dict_untranslated()

In [ ]:
# 构建对比
pf.create_contrast()

In [ ]:

def find_all(data:list, failed_text:list, jp_chs:dict):
    record = 0
    cnt = 0                         # 当前行
    start = False
    for line in data:
        #START FIXME 替换目标文本
        if pf.has_jp(line) and line[0] not in "@*;\t /[":
            key = line[:-1]
            if key in jp_chs and jp_chs[key]:
                record += 1
                data[cnt] = data[cnt].replace(key, jp_chs[key])
            else:
                failed_text.append(key+'\n')
        cnt += 1
        # END 
    return record

pf.replace_all(find_all, encoding, output_encoding)

In [ ]:
'杏那を逃がす'.encode('cp932').decode('gbk')

In [ ]:

new_jp_chs = dict()
jp_chs = pf.open_json('intermediate_file/jp_chs copy 2.json')
for line in jp_all:
    temp = line.replace('[','')
    temp = temp.replace(']','')
    temp = temp.replace('　','')
    temp = temp.replace('\\n','')
    temp = temp.replace('\\f','')
    temp_ans = jp_chs[temp]
    new_jp_chs[line] = temp_ans
    if line[0] == '[':
        _p = temp_ans.find('「')
        if _p != -1:
            temp_ans = '['+temp_ans[:_p]+']'+temp_ans[_p:]
            new_jp_chs[line] = temp_ans
            continue
        _p = temp_ans.find('（')
        if _p != -1:
            temp_ans = '['+temp_ans[:_p]+']'+temp_ans[_p:]
            new_jp_chs[line] = temp_ans
            continue
        _p = temp_ans.find('『')
        if _p != -1:
            temp_ans = '['+temp_ans[:_p]+']'+temp_ans[_p:]
            new_jp_chs[line] = temp_ans
            continue
        
pf.save_json('intermediate_file/jp_chs.json', new_jp_chs)

In [14]:
def strB2Q(ustring, exclude=()):
    """半角转全角"""
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 32:                                 #半角空格直接转化                  
            inside_code = 12288
        elif inside_code >= 32 and inside_code <= 126:        #半角字符（除空格）根据关系转化
            inside_code += 65248
        if uchar in exclude:
            rstring += uchar
        else:
            rstring += chr(inside_code)
    return rstring
strB2Q('[神夢]「啊，是啊…是的是的…嗯，嗯，嗯-嗯…！」\\f')

'［神夢］「啊，是啊…是的是的…嗯，嗯，嗯－嗯…！」＼ｆ'